In [64]:
# Selenium imports
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup

from time import sleep

C:\Users\Felipe\AppData\Local\Temp\ipykernel_4164\2609162591.py:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [65]:
# General Helpers

headers = {
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "http://thewebsite.com",
    "Connection": "keep-alive"}

def price_multiple_replace(text, dict=None):
    """
    Multiple regex replacements (to replace in key, replacement in value) 
    """
    if not dict:
        dict = {"," : ".", "." : ""}
        
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))
    
    # For each match, look-up corresponding value in dictionary
    return float(regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text))

# Argenprop

In [66]:
def scrape_argenprop(url_ap):
    
    result_ap = requests.get(url_ap, headers=headers)

    soup_ap = BeautifulSoup(result_ap.text, 'lxml')

    all_data_ap = []

    # Links
    base_link = 'https://www.argenprop.com'
    div_container = soup_ap.find_all('div',{'class':'listing__item'})
    links_ap = []
    for div in div_container:
        a_link = div.find_all('a', href=True)
        last_link = a_link[0]['href']
        links_ap.append(base_link+last_link)
    all_data_ap.append(links_ap)


    # Service
    all_data_ap.append(['argenprop']*len(links_ap))

    # City
    city_ap = soup_ap.find_all('p', {'class':'card__title--primary show-mobile'})
    city_ap = [c.text for c in city_ap]
    all_data_ap.append(city_ap)

    # Adress
    address_ap = soup_ap.find_all(['h2','p'], {'class':'card__address'})
    address_ap = [c.text.strip() for c in address_ap]
    address_ap = [re.findall('.+', str(c))[0] for c in address_ap]
    all_data_ap.append(address_ap)

    # Property Type
    # property_type_ap = soup_ap.find_all('p', {'class':'card__title--primary hide-mobile'})
    # property_type_ap = [c.text.split(' ')[0] for c in property_type_ap]
    if 'casa' in url_ap:
        property_type_ap = ['Casa']*len(links_ap)
    else:
        property_type_ap = ['Departamento']*len(links_ap)
    all_data_ap.append(property_type_ap)

    # Contract type
    contract_type_ap = soup_ap.find_all('p', {'class':'card__title--primary hide-mobile'})
    contract_type_ap = [c.text.split(' ')[2] for c in contract_type_ap]
    all_data_ap.append(contract_type_ap)


    # Rooms and Squared Meters
    ul_container = soup_ap.find_all('ul', {'class':'card__main-features'})
    rooms_ap = []
    squared_meters_ap = []
    for ul in ul_container:
        
        amb = re.findall('(\d+)\s(?:ambiente|dorm\.)', str(ul.find_all('span')))
        if len(amb)>0:
            rooms_ap.append(int(amb[0]))
        else:
            rooms_ap.append(1)
            
        m2 = re.findall('(\d+)\s*m²', str(ul.find_all('span')))
        if len(m2)>0:
            squared_meters_ap.append(int(m2[0]))
        else:
            squared_meters_ap.append(0)
    all_data_ap.append(rooms_ap)
    all_data_ap.append(squared_meters_ap)


    # Price and currency type
    buy_rent_currency_ap = []
    buy_rent_price_ap = []

    expensas_currency_ap = []
    expensas_price_ap = []

    all_currency_and_price_ap = soup_ap.find_all('p', {'class':'card__price'})
    all_currency_and_price_ap = [c.text for c in all_currency_and_price_ap]
    all_currency_and_price_ap = [re.findall('(USD|\$)\s?(\d+(?:[,.]\d+)*)', str(c)) for c in all_currency_and_price_ap]

    for cp in all_currency_and_price_ap:
        if len(cp)>0:
            buy_rent_currency_ap.append(cp[0][0])
            buy_rent_price_ap.append(price_multiple_replace(cp[0][1]))
            if len(cp)>1:
                expensas_currency_ap.append(cp[1][0])
                expensas_price_ap.append(price_multiple_replace(cp[1][1]))
            else:
                expensas_currency_ap.append('consultar_precio')
                expensas_price_ap.append(0)
        else:
            buy_rent_currency_ap.append('consultar_precio')
            buy_rent_price_ap.append(0)
            expensas_currency_ap.append('consultar_precio')
            expensas_price_ap.append(0)
            
    all_data_ap.append(buy_rent_currency_ap)
    all_data_ap.append(expensas_currency_ap)
    all_data_ap.append(buy_rent_price_ap)
    all_data_ap.append(expensas_price_ap)

    return all_data_ap

In [67]:
all_url_ap = [  'https://www.argenprop.com/departamento-venta-pagina-',
                'https://www.argenprop.com/departamento-alquiler-y-alquiler-temporal-pagina-',
                'https://www.argenprop.com/casa-venta-pagina-',
                'https://www.argenprop.com/casa-alquiler-y-alquiler-temporal-pagina-']

cantidad_paginas_a_scrapear = {
                                'departamentos venta':2,
                                'departamentos alquiler':2,
                                'casas venta':2,
                                'casas alquiler':2,
                                }

all_columns_ap = []
for i_url, url_ap in enumerate(all_url_ap): # iterate general URLs
    cant_pag = list(cantidad_paginas_a_scrapear.values())[i_url]
    categorie = list(cantidad_paginas_a_scrapear.keys())[i_url]
    for pag in range(cant_pag): # iterate paginas
        print(f'Scraping {categorie}, pag. {pag+1} of {cant_pag}.')
        
        url_ap = url_ap+str(pag+1)
        
        all_columns_ap.append(scrape_argenprop(url_ap))

Scraping departamentos venta, pag. 1 of 2.
Scraping departamentos venta, pag. 2 of 2.
Scraping departamentos alquiler, pag. 1 of 2.
Scraping departamentos alquiler, pag. 2 of 2.
Scraping casas venta, pag. 1 of 2.
Scraping casas venta, pag. 2 of 2.
Scraping casas alquiler, pag. 1 of 2.
Scraping casas alquiler, pag. 2 of 2.


# Mercado Libre

In [68]:
# Helpers
def get_ml_expensas_price(link_expensas):
    
    """
    For Mercado Libre
    """
    expensas_price = []

    for l_expensas in link_expensas:
        result = requests.get(l_expensas, headers=headers)
        soup = BeautifulSoup(result.text, 'lxml')
        try:
            exp = soup.find('p', {'class':'ui-pdp-color--GRAY ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-maintenance-fee-ltr'}).text
        except:
            exp = '0'
        expensas_price.append(exp)

    expensas_price = [re.findall('\d+(?:[,.]\d+)*', ex)[0] for ex in expensas_price]
    expensas_price = [int(price_multiple_replace(c)) for c in expensas_price]
    
    return expensas_price

In [69]:
def scrape_mercadolibre(url_ml, scrap_expensas_ML = True):

    result_ml = requests.get(url_ml, headers=headers)

    soup_ml = BeautifulSoup(result_ml.text, 'lxml')
        
    all_data_ml = []

    # Links
    div_container = soup_ml.find_all('div',{'class':'ui-search-result__wrapper shops__result-wrapper'})
    links_ml = []
    for div in div_container:
        a_link = div.find_all('a', href=True)
        link = a_link[0]['href']
        links_ml.append(link)
    all_data_ml.append(links_ml)

    # Service
    all_data_ml.append(['mercado_libre']*len(links_ml))

    # City
    city_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__location shops__items-group-details'})
    city_ml = [c.text for c in city_ml]
    all_data_ml.append(city_ml)


    # Adress
    address_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__location shops__items-group-details'})
    address_ml = [c.text.split(',')[0] for c in address_ml]
    all_data_ml.append(address_ml)

    # Property type
    # property_type_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__subtitle shops__items-group-details'})
    # property_type_ml = [c.text.split(' en ')[0] for c in property_type_ml]
    if 'casa' in url_ml:
        property_type_ml = ['Casa']*len(links_ml)
    else:
        property_type_ml = ['Departamento']*len(links_ml)
    all_data_ml.append(property_type_ml)

    # Contract type
    contract_type_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__subtitle shops__items-group-details'})
    contract_type_ml = [c.text.split(' en ')[-1] for c in contract_type_ml]
    all_data_ml.append(contract_type_ml)

    # Rooms and Squared Meters
    squared_meters = []
    rooms = []
    squared_meters_and_rooms = soup_ml.find_all('ul', {'class':'ui-search-card-attributes ui-search-item__group__element shops__items-group-details'})
    squared_meters_and_rooms = [sm_and_r.find_all('li', {'class':'ui-search-card-attributes__attribute'}) for sm_and_r in squared_meters_and_rooms]

    for sm_and_r in squared_meters_and_rooms:
        sm_and_r = [smr.text for smr in sm_and_r]
        if len(sm_and_r)==2:
            squared_meters.append(sm_and_r[0])
            rooms.append(sm_and_r[1])
        else:
            if 'm²' in sm_and_r[0]:
                squared_meters.append(sm_and_r[0])
                rooms.append('0')
            else:
                squared_meters.append('0')
                rooms.append(sm_and_r[0])
                
    squared_meters = [int(price_multiple_replace(sm.split(' ')[0])) for sm in squared_meters]
    rooms = [int(price_multiple_replace(r.split(' ')[0])) for r in rooms]
    all_data_ml.append(rooms)
    all_data_ml.append(squared_meters)


    # All Currency type
    buy_rent_currency_ml = soup_ml.find_all('span', {'class':'price-tag-symbol'})
    buy_rent_currency_ml = [c.text.replace('U$S', 'USD') for c in buy_rent_currency_ml]
    buy_rent_currency_ml = buy_rent_currency_ml[len(buy_rent_currency_ml)-len(links_ml):] # Eliminate those with no link
    expensas_currency_ml = ['$']*len(links_ml)
    all_data_ml.append(buy_rent_currency_ml)
    all_data_ml.append(expensas_currency_ml)


    # Buy Rent Price
    buy_rent_price_ml = []
    buy_rent_price_ml = soup_ml.find_all('span', {'class':'price-tag-fraction'})
    buy_rent_price_ml = [c.text for c in buy_rent_price_ml]
    buy_rent_price_ml = [int(price_multiple_replace(c)) for c in buy_rent_price_ml]
    buy_rent_price_ml = buy_rent_price_ml[len(buy_rent_price_ml)-len(links_ml):] # Eliminate those with no link
    all_data_ml.append(buy_rent_price_ml)


    # Expensas Price
    if scrap_expensas_ML:
        expensas_price = get_ml_expensas_price(links_ml)
    else:
        expensas_price = [0]*len(links_ml)
    all_data_ml.append(expensas_price)
    
    return all_data_ml

In [71]:
all_url_ml = ['https://inmuebles.mercadolibre.com.ar/departamentos/venta/_Desde_NRO_PAGINA_NoIndex_True',
              'https://inmuebles.mercadolibre.com.ar/departamentos/alquiler-temporario/_Desde_NRO_PAGINA_NoIndex_True',
              'https://inmuebles.mercadolibre.com.ar/casas/venta/_Desde_NRO_PAGINA_NoIndex_True',
              'https://inmuebles.mercadolibre.com.ar/casas/alquiler-temporario/_Desde_NRO_PAGINA_NoIndex_True']

cantidad_paginas_a_scrapear = {
                                'departamentos venta':2,
                                'departamentos alquiler':2,
                                'casas venta':2,
                                'casas alquiler':2,
                                }


all_columns_ml = []
for i_url, url_ml in enumerate(all_url_ml): # iterate general URLs
    cant_pag = list(cantidad_paginas_a_scrapear.values())[i_url]
    categorie = list(cantidad_paginas_a_scrapear.keys())[i_url]
    for pag in range(cant_pag): # iterate paginas
        print(f'Scraping {categorie}, pag. {pag+1} of {cant_pag}.')
        
        pag_counter = (48*pag)+1
        
        url_ml = url_ml.replace('NRO_PAGINA',str(pag_counter)) #define page number
        
        all_columns_ml.append(scrape_mercadolibre(url_ml))

Scraping departamentos venta, pag. 1 of 2.
Scraping departamentos venta, pag. 2 of 2.
Scraping departamentos alquiler, pag. 1 of 2.
Scraping departamentos alquiler, pag. 2 of 2.
Scraping casas venta, pag. 1 of 2.
Scraping casas venta, pag. 2 of 2.
Scraping casas alquiler, pag. 1 of 2.
Scraping casas alquiler, pag. 2 of 2.


# Dataframe

In [72]:
# Combine each of the 12 (city, price, etc) columns found in each of the N pages scraped
amount_columns = 12

ml_combined = [ [item for sublist in all_columns_ml for item in sublist[i]] for i in range(amount_columns)]
ap_combined = [ [item for sublist in all_columns_ap for item in sublist[i]] for i in range(amount_columns)]

# Combine every scrapped web
all_columns = []
for i in range(amount_columns):
    all_columns.append(ml_combined[i]+ap_combined[i])
    

col_names = ['link','service','adress','city','property_type','contract_type','rooms','squared_meters','buy_rent_currency','expensas_currency','buy_rent_price','expensas_price']
df = pd.DataFrame(all_columns).T
df.columns = col_names

## Save

In [75]:
# Checkpoint
# df.to_csv('scraped_df.csv', index=False)

## Load

In [76]:
# Checkpoint
df = pd.read_csv('scraped_df.csv')

# Zonaprop

In [38]:
url_zp = 'https://www.zonaprop.com.ar/inmuebles-alquiler.html'

In [39]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\Felipe\AppData\Local\Temp\ipykernel_4164\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [40]:
driver.get(url_zp)

In [56]:
all_data_zp = []

# Links
start_link_zp = 'https://www.zonaprop.com.ar'
links_zp = driver.find_elements(By.XPATH, '//*/div[@data-to-posting]')
links_zp = [start_link_zp+l.get_attribute('data-to-posting') for l in links_zp]

# Service
all_data_zp.append(['zona_prop']*len(links_zp))

# City
city_zp = driver.find_elements(By.XPATH, '//*/div[@data-qa="POSTING_CARD_LOCATION"]')
city_zp = [c.text for c in city_zp]

# Adress
adress_zp = driver.find_elements(By.XPATH, '//*/div[@class="sc-ge2uzh-0 bzGYzE"]')
adress_zp = [c.text for c in adress_zp]

# Property type
if 'casa' in url_ml:
    property_type_zp = ['Casa']*len(links_zp)
else:
    property_type_zp = ['Departamento']*len(links_zp)
all_data_zp.append(property_type_zp)

# SQL

,link,service,adress,city,property_type,contract_type,rooms,squared_meters,buy_rent_currency,expensas_currency,buy_rent_price,expensas_price
0,https://departamento.mercadolibre.com.ar/MLA-1...,mercado_libre,"H. Puyrredon Al 800, Caballito, Capital Federal",H. Puyrredon Al 800,Departamento,venta,4,72,USD,$,124900.0,18000.0
1,https://departamento.mercadolibre.com.ar/MLA-1...,mercado_libre,"San Luis Al 1800, Centro, Mar Del Plata, Bs.as...",San Luis Al 1800,Departamento,venta,2,40,USD,$,65900.0,7000.0
2,https://departamento.mercadolibre.com.ar/MLA-1...,mercado_libre,"Superí 2300, Belgrano, Capital Federal",Superí 2300,Departamento,venta,4,69,USD,$,115000.0,15000.0
3,https://departamento.mercadolibre.com.ar/MLA-1...,mercado_libre,"Av. Belgrano Al 1400, Monserrat, Capital Federal",Av. Belgrano Al 1400,Departamento,venta,3,80,USD,$,139000.0,14000.0
4,https://departamento.mercadolibre.com.ar/MLA-1...,mercado_libre,"Olazabal 5100, Villa Urquiza, Capital Federal",Olazabal 5100,Departamento,venta,2,42,USD,$,94900.0,14000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
539,https://www.argenprop.com/casa-en-alquiler-en-...,argenprop,"Santa Barbara, Countries y Barrios Cerrados en...",Sta Barbara Corr Bancalari 3900,Casa,Alquiler,5,400,USD,consultar_precio,2600.0,0.0
540,https://www.argenprop.com/casa-en-alquiler-en-...,argenprop,"Mayling Club De Campo, Countries y Barrios Cer...",Mayling Club de Campo Chubut al 400,Casa,Alquiler,4,836,USD,consultar_precio,4500.0,0.0
541,https://www.argenprop.com/casa-en-alquiler-en-...,argenprop,"Jockey, San Isidro",3 de Febrero al 2400,Casa,Alquiler,3,370,USD,consultar_precio,3600.0,0.0
542,https://www.argenprop.com/casa-en-alquiler-en-...,argenprop,"Laguna Del Sol, Countries y Barrios Cerrados e...","Laguna del Sol , Escalada 1200",Casa,Alquiler,4,220,USD,consultar_precio,2300.0,0.0
